In [25]:
pip install langgraph langchain-groq langchain-core requests

In [37]:
import os
from typing import Literal, TypedDict
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
import requests
import json
import re
# Set your API keys
os.environ["GROQ_API_KEY"] = ""
os.environ["WEATHER_API_KEY"] = ""

# Define the state
class AgentState(TypedDict):
    messages: list
    next_agent: Literal["weather_agent", "python_agent", "coordinator", "end"] = "coordinator"

# Initialize Groq LLM
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.1
)

# Weather Agent
def weather_agent(state: AgentState):
    """Agent that fetches weather data from weatherapi.com"""
    try:
        # Extract location using more robust pattern matching
        last_message = state["messages"][-1].content if state["messages"] else ""

        # Look for location patterns
        location_patterns = [
            r"weather in (.+?)(?:\?|$)",
            r"weather for (.+?)(?:\?|$)",
            r"temperature in (.+?)(?:\?|$)",
            r"forecast for (.+?)(?:\?|$)"
        ]

        location = None
        for pattern in location_patterns:
            match = re.search(pattern, last_message, re.IGNORECASE)
            if match:
                location = match.group(1).strip()
                break

        # If no pattern matched, try to extract the last word as location
        if not location:
            words = last_message.split()
            if len(words) > 2:
                location = words[-1].rstrip('?.!')

        # Default to London if no location found
        if not location:
            location = "London"

        # Call WeatherAPI
        api_key = os.environ["WEATHER_API_KEY"]
        url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={location}&aqi=no"

        response = requests.get(url)
        data = response.json()

        if response.status_code == 200:
            weather_info = {
                "location": data["location"]["name"],
                "country": data["location"]["country"],
                "temperature": data["current"]["temp_c"],
                "condition": data["current"]["condition"]["text"],
                "humidity": data["current"]["humidity"],
                "wind_speed": data["current"]["wind_kph"]
            }

            response_text = f"""Weather in {weather_info['location']}, {weather_info['country']}:
- Temperature: {weather_info['temperature']}°C
- Condition: {weather_info['condition']}
- Humidity: {weather_info['humidity']}%
- Wind Speed: {weather_info['wind_speed']} km/h"""

        else:
            response_text = f"Error fetching weather data: {data.get('error', {}).get('message', 'Unknown error')}"

    except Exception as e:
        response_text = f"Error in weather agent: {str(e)}"

    return {
        "messages": [AIMessage(content=response_text)],
        "next_agent": "end"
    }

# Python REPL Agent
def python_agent(state: AgentState):
    """Agent that executes Python code and returns results"""
    try:
        # Extract the user's request
        last_message = state["messages"][-1].content if state["messages"] else ""

        # Use LLM to generate appropriate Python code
        prompt = f"""The user asked: "{last_message}"

Please generate appropriate Python code to fulfill this request.
Return ONLY the Python code without any explanations or markdown formatting.
If the request is not suitable for Python code execution, return an empty string."""

        response = llm.invoke([HumanMessage(content=prompt)])
        code = response.content.strip()

        # Remove markdown code blocks if present
        if "```python" in code:
            code = code.split("```python")[1].split("```")[0].strip()
        elif "```" in code:
            code = code.split("```")[1].split("```")[0].strip()

        # If no code was generated or request not suitable for Python
        if not code or "I cannot" in code or "I can't" in code:
            return {
                "messages": [AIMessage(content="I'm not able to generate appropriate Python code for this request.")],
                "next_agent": "end"
            }

        # Execute the code safely (with restrictions)
        allowed_modules = ["math", "datetime", "random", "json", "collections"]
        exec_globals = {}

        for module in allowed_modules:
            try:
                exec_globals[module] = __import__(module)
            except ImportError:
                pass

        # Execute the code and capture output
        try:
            # Redirect stdout to capture print statements
            from io import StringIO
            import sys

            old_stdout = sys.stdout
            sys.stdout = captured_output = StringIO()

            exec(code, exec_globals)

            sys.stdout = old_stdout
            result = captured_output.getvalue()

            if not result:
                result = "Code executed successfully (no output)"

        except Exception as e:
            result = f"Error executing code: {str(e)}"

        response_text = f"I executed the following Python code:\n```python\n{code}\n```\nResult:\n{result}"

    except Exception as e:
        response_text = f"Error in Python agent: {str(e)}"

    return {
        "messages": [AIMessage(content=response_text)],
        "next_agent": "end"
    }

# Coordinator Agent
def coordinator_agent(state: AgentState):
    """Agent that decides which specialized agent to invoke"""
    last_message = state["messages"][-1].content if state["messages"] else ""

    # Use LLM to decide which agent to call with better prompting
    prompt = f"""Analyze the user's request and decide which agent should handle it:

User request: "{last_message}"

Available agents:
1. weather_agent - for weather-related queries (temperature, forecast, weather conditions, climate)
2. python_agent - for calculations, data processing, math problems, or any request that can be solved with code

Consider these examples:
- "What's the weather in Paris?" → weather_agent
- "Calculate 5 * 8" → python_agent
- "What's the temperature in Tokyo?" → weather_agent
- "Write code to calculate factorial of 5" → python_agent
- "How humid is it in London?" → weather_agent
- "Solve 2x + 5 = 15" → python_agent

Respond with ONLY the name of the appropriate agent: either "weather_agent" or "python_agent"
If the request doesn't match either, respond with "end"."""

    response = llm.invoke([HumanMessage(content=prompt)])
    next_agent = response.content.strip().lower()

    # Validate the response
    if next_agent not in ["weather_agent", "python_agent"]:
        next_agent = "end"

    return {
        "messages": state["messages"],
        "next_agent": next_agent
    }

# Build the graph
def build_agent_graph():
    workflow = StateGraph(AgentState)

    # Add nodes
    workflow.add_node("coordinator", coordinator_agent)
    workflow.add_node("weather_agent", weather_agent)
    workflow.add_node("python_agent", python_agent)

    # Set entry point
    workflow.set_entry_point("coordinator")

    # Add edges
    workflow.add_conditional_edges(
        "coordinator",
        lambda state: state["next_agent"],
        {
            "weather_agent": "weather_agent",
            "python_agent": "python_agent",
            "end": END
        }
    )

    workflow.add_edge("weather_agent", END)
    workflow.add_edge("python_agent", END)

    return workflow.compile()

# Create the graph
agent_graph = build_agent_graph()

# Function to run the agent system
def run_agent_system(user_input: str):
    """Run the agent system with user input"""
    initial_state = {
        "messages": [HumanMessage(content=user_input)],
        "next_agent": "coordinator"
    }

    result = agent_graph.invoke(initial_state)
    return result["messages"][-1].content

# Example usage
if __name__ == "__main__":
    # Test examples
    test_queries = [
        "What's the weather in Sydney?",
        "Can you calculate 5 * 8 for me?",
        "What's the temperature in Tokyo?",
        "Write Python code to calculate factorial of 5",
        "How humid is it in London?",
        "Solve for x: 2x + 5 = 15"
    ]

    for query in test_queries:
        print(f"\nUser: {query}")
        response = run_agent_system(query)
        print(f"Response: {response}")
        print("-" * 50)


User: What's the weather in Sydney?
Response: Weather in Sydney, Australia:
- Temperature: 14.0°C
- Condition: Light rain
- Humidity: 77%
- Wind Speed: 28.8 km/h
--------------------------------------------------

User: Can you calculate 5 * 8 for me?
Response: I executed the following Python code:
```python
print(5 * 8)
```
Result:
40

--------------------------------------------------

User: What's the temperature in Tokyo?
Response: Weather in Tokyo, Japan:
- Temperature: 23.3°C
- Condition: Partly cloudy
- Humidity: 61%
- Wind Speed: 24.8 km/h
--------------------------------------------------

User: Write Python code to calculate factorial of 5
Response: I executed the following Python code:
```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

print(factorial(5))
```
Result:
120

--------------------------------------------------

User: How humid is it in London?
Response: Weather in London, United Kingdom:
- Temperature: 11.3°C

In [47]:
import os
from typing import Literal, TypedDict
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
import requests
import json
import re

os.environ["GROQ_API_KEY"] = ""
os.environ["WEATHER_API_KEY"] = ""
# Define the state
class AgentState(TypedDict):
    messages: list
    next_agent: Literal["weather_agent", "python_agent", "coordinator", "end"] = "coordinator"

# Initialize Groq LLM
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.1
)

# Weather Agent
def weather_agent(state: AgentState):
    """Agent that fetches weather data from weatherapi.com"""
    try:
        # Extract location using more robust pattern matching
        last_message = state["messages"][-1].content if state["messages"] else ""

        # Look for location patterns
        location_patterns = [
            r"weather in (.+?)(?:\?|$)",
            r"weather for (.+?)(?:\?|$)",
            r"temperature in (.+?)(?:\?|$)",
            r"forecast for (.+?)(?:\?|$)"
        ]

        location = None
        for pattern in location_patterns:
            match = re.search(pattern, last_message, re.IGNORECASE)
            if match:
                location = match.group(1).strip()
                break

        # If no pattern matched, try to extract the last word as location
        if not location:
            words = last_message.split()
            if len(words) > 2:
                location = words[-1].rstrip('?.!')

        # Default to London if no location found
        if not location:
            location = "London"

        # Call WeatherAPI
        api_key = os.environ["WEATHER_API_KEY"]
        url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={location}&aqi=no"

        response = requests.get(url)
        data = response.json()

        if response.status_code == 200:
            weather_info = {
                "location": data["location"]["name"],
                "country": data["location"]["country"],
                "temperature": data["current"]["temp_c"],
                "condition": data["current"]["condition"]["text"],
                "humidity": data["current"]["humidity"],
                "wind_speed": data["current"]["wind_kph"]
            }

            response_text = f"""Weather in {weather_info['location']}, {weather_info['country']}:
- Temperature: {weather_info['temperature']}°C
- Condition: {weather_info['condition']}
- Humidity: {weather_info['humidity']}%
- Wind Speed: {weather_info['wind_speed']} km/h"""

        else:
            response_text = f"Error fetching weather data: {data.get('error', {}).get('message', 'Unknown error')}"

    except Exception as e:
        response_text = f"Error in weather agent: {str(e)}"

    return {
        "messages": [AIMessage(content=response_text)],
        "next_agent": "end"
    }

# Python REPL Agent
def python_agent(state: AgentState):
    """Agent that executes Python code and returns results"""
    try:
        # Extract the user's request
        last_message = state["messages"][-1].content if state["messages"] else ""

        # Use LLM to generate appropriate Python code
        prompt = f"""The user asked: "{last_message}"

Please generate appropriate Python code to fulfill this request.
Return ONLY the Python code without any explanations or markdown formatting.
If the request is not suitable for Python code execution, return an empty string."""

        response = llm.invoke([HumanMessage(content=prompt)])
        code = response.content.strip()

        # Remove markdown code blocks if present
        if "```python" in code:
            code = code.split("```python")[1].split("```")[0].strip()
        elif "```" in code:
            code = code.split("```")[1].split("```")[0].strip()

        # If no code was generated or request not suitable for Python
        if not code or "I cannot" in code or "I can't" in code:
            return {
                "messages": [AIMessage(content="I'm not able to generate appropriate Python code for this request.")],
                "next_agent": "end"
            }

        # Execute the code safely (with restrictions)
        allowed_modules = ["math", "datetime", "random", "json", "collections"]
        exec_globals = {}

        for module in allowed_modules:
            try:
                exec_globals[module] = __import__(module)
            except ImportError:
                pass

        # Execute the code and capture output
        try:
            # Redirect stdout to capture print statements
            from io import StringIO
            import sys

            old_stdout = sys.stdout
            sys.stdout = captured_output = StringIO()

            exec(code, exec_globals)

            sys.stdout = old_stdout
            result = captured_output.getvalue()

            if not result:
                result = "Code executed successfully (no output)"

        except Exception as e:
            result = f"Error executing code: {str(e)}"

        response_text = f"I executed the following Python code:\n```python\n{code}\n```\nResult:\n{result}"

    except Exception as e:
        response_text = f"Error in Python agent: {str(e)}"

    return {
        "messages": [AIMessage(content=response_text)],
        "next_agent": "end"
    }

# Coordinator Agent
def coordinator_agent(state: AgentState):
    """Agent that decides which specialized agent to invoke"""
    last_message = state["messages"][-1].content if state["messages"] else ""

    # Use LLM to decide which agent to call with better prompting
    prompt = f"""Analyze the user's request and decide which agent should handle it:

User request: "{last_message}"

Available agents:
1. weather_agent - for weather-related queries (temperature, forecast, weather conditions, climate)
2. python_agent - for calculations, data processing, math problems, or any request that can be solved with code

Consider these examples:
- "What's the weather in Paris?" → weather_agent
- "Calculate 5 * 8" → python_agent
- "What's the temperature in Tokyo?" → weather_agent
- "Write code to calculate factorial of 5" → python_agent
- "How humid is it in London?" → weather_agent
- "Solve 2x + 5 = 15" → python_agent

Respond with ONLY the name of the appropriate agent: either "weather_agent" or "python_agent"
If the request doesn't match either, respond with "end"."""

    response = llm.invoke([HumanMessage(content=prompt)])
    next_agent = response.content.strip().lower()

    # Validate the response
    if next_agent not in ["weather_agent", "python_agent"]:
        next_agent = "end"

    return {
        "messages": state["messages"],
        "next_agent": next_agent
    }

# Build the graph
def build_agent_graph():
    workflow = StateGraph(AgentState)

    # Add nodes
    workflow.add_node("coordinator", coordinator_agent)
    workflow.add_node("weather_agent", weather_agent)
    workflow.add_node("python_agent", python_agent)

    # Set entry point
    workflow.set_entry_point("coordinator")

    # Add edges
    workflow.add_conditional_edges(
        "coordinator",
        lambda state: state["next_agent"],
        {
            "weather_agent": "weather_agent",
            "python_agent": "python_agent",
            "end": END
        }
    )

    workflow.add_edge("weather_agent", END)
    workflow.add_edge("python_agent", END)

    return workflow.compile()

# Create the graph
agent_graph = build_agent_graph()

# Function to run the agent system
def run_agent_system(user_input: str):
    """Run the agent system with user input"""
    initial_state = {
        "messages": [HumanMessage(content=user_input)],
        "next_agent": "coordinator"
    }

    result = agent_graph.invoke(initial_state)
    return result["messages"][-1].content


# Example usage
if __name__ == "__main__":


    print("\n📋 Agent Descriptions:")
    print("• coordinator_agent: Analyzes user input and routes to appropriate agent")
    print("• weather_agent: Fetches weather data from weatherapi.com")
    print("• python_agent: Executes Python code for calculations")
    print("• router: Conditional routing based on coordinator decision")

    print("\n🧪 Testing the system:")
    print("=" * 50)

    # Test examples
    test_queries = [
        "What's the weather in Sydney?",
        "Can you calculate 5 * 8 for me?",
        "What's the temperature in Tokyo?",
        "Write Python code to calculate factorial of 5",
        "How humid is it in London?",
        "Solve for x: 2x + 5 = 15"
    ]

    for query in test_queries:
        print(f"\n👤 User: {query}")
        response = run_agent_system(query)
        print(f"🤖 Response: {response}")
        print("-" * 80)


📋 Agent Descriptions:
• coordinator_agent: Analyzes user input and routes to appropriate agent
• weather_agent: Fetches weather data from weatherapi.com
• python_agent: Executes Python code for calculations
• router: Conditional routing based on coordinator decision

🧪 Testing the system:

👤 User: What's the weather in Sydney?
🤖 Response: Weather in Sydney, Australia:
- Temperature: 13.4°C
- Condition: Light rain shower
- Humidity: 77%
- Wind Speed: 28.8 km/h
--------------------------------------------------------------------------------

👤 User: Can you calculate 5 * 8 for me?
🤖 Response: I executed the following Python code:
```python
print(5 * 8)
```
Result:
40

--------------------------------------------------------------------------------

👤 User: What's the temperature in Tokyo?
🤖 Response: Weather in Tokyo, Japan:
- Temperature: 23.8°C
- Condition: Clear
- Humidity: 46%
- Wind Speed: 24.8 km/h
--------------------------------------------------------------------------------

👤